<a href="https://colab.research.google.com/github/mikancode/daily-coding/blob/main/20251230-corp-url-finder/01_search_official_url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests

In [3]:
from google.colab import userdata
#userdata.get('secretName')

In [ ]:
import json

url = "https://trial.api.info.gbiz.go.jp/hojin/v2/hojin"

params = {
    "page":1,
    "limit":1000,
    "metadata_flg":"false",
    "name":"paiza"
}

headers = {"X-hojinInfo-api-token": userdata.get('GBIZ_API_KEY')}


try:
    response = requests.get(url, params=params, headers=headers, timeout=10)  # タイムアウト設定
    response.raise_for_status()  # ステータスコードが200番台以外の場合に例外を発生
    data = response.json()  # レスポンスボディをJSONとして解析
    # dataを使った処理...
    display(data)

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

In [ ]:
hji = data['hojin-infos']
print(hji)
full_url = f"{url}/{hji[0]['corporate_number']}"

print(full_url)

params = {
    "metadata_flg":"true"
}

headers = {"X-hojinInfo-api-token": userdata.get('GBIZ_API_KEY')}


try:
    response = requests.get(full_url, params=params, headers=headers, timeout=10)  # タイムアウト設定
    response.raise_for_status()  # ステータスコードが200番台以外の場合に例外を発生
    data = response.json()  # レスポンスボディをJSONとして解析
    # dataを使った処理...
    display(data)

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")